In [1]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/neg_LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv") %>%
    arrange(site)
#setorder(Centroids, site)#warning! This sets order based on site, and then lat/lon. So the table is not alphabetical by site, but that's fine as long as all of the "sampled_reef" vectors reflect this, so that reef_sizes, distance, and sampled_reefs match up by row/col index 
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- unique(fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")[
    , .(site, year=end_year, prop_anem_samp=total_prop_hab_sampled_anems_tidied)][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , prop_anem_samp := sum(prop_anem_samp), by=c("site", "year")], by=c("site", "year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & year %in% c(2012, 2013, 2014), prop_anem_samp :=1][site=="Caridad Proper" & year %in% c(2013, 2014), prop_anem_samp :=1]
#add in the numbers of particles seeded at each site
SeededParticles <- fread("~/oceanography/ROMS/data/Particles_Per_Release_Site_Renamed.csv")
setnames(SeededParticles,c("source", "daily_particles_released")) 
#DateJoin <- SeededParticles[DateJoin, on="source"][, particles_released_daily := as.numeric(particles_released_daily)] 

#make vectors defining sites we didn't sample, but that are in the model, and the sandflats specifically 
unsampled_sites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 
sand_flats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6") 
unrealistic_sources <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan") 
#make the constant inputs for the kernel fitting function
#distance matrix using the centroids with combined Magbangon
### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
#setorder(Area, site)
reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel 
SiteIndexBioPhys <- unique(Centroids %>% arrange(site), by="site")[site != "CAI"][, index := .I] #add the row number as the unique site index, take CAI out for biophysical likelihood function

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
Abundance <- unique(Abundance[site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
            , num_females := sum(num_females), by=c("site", "year")], by=c("site", "year"))
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0


SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this
#quick check that all components are in the same, alphabetical order
sum(which(SiteIndex$site==Area$site)==FALSE) #needs to be 0!! sites have to be in the same order
sum(which(Area$site==Centroids$site)==FALSE) #needs to be 0!! sites have to be in the same order

#read in genetic parentage matrices for each time frame MATCHING DIMENSIONS WITH BIOPHYSICAL FOR BIOPHYSICAL LIKELIHOOD- NOT KERNEL FITTING
GenMat2012_4 <- fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2012-14ForROMSComp.csv")
GenMat2012 <- fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2012ForROMSComp.csv")
GenMat2013 <- fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2013ForROMSComp.csv")
GenMat2014 <- fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2014ForROMSComp.csv")
GenMatNEM <- fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrixNEM2012-14ForROMSComp.csv")
GenMatSWM <- fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrixSWM2012-14ForROMSComp.csv")

#read in genetic parentage matrices for each time frame FOR KERNEL FITTING
KernelGenMat2012_4 <- fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv")
KernelGenMat2012 <- fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv")
KernelGenMat2013 <- fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv")
KernelGenMat2014 <- fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv")
KernelGenMatNEM <- fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv")
KernelGenMatSWM <- fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv")

#read in biophysical data for biophysical likelihood
FullBiophysMatNorm <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2012-14ForROMSComp.csv")
AnnualBiophysMatNorm2012 <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2012ForROMSComp.csv")
AnnualBiophysMatNorm2013 <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2013ForROMSComp.csv")
AnnualBiophysMatNorm2014 <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2014ForROMSComp.csv")
MonsoonBiophysMatNormNEM <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrixNEM2012-14ForROMSComp.csv")
MonsoonBiophysMatNormSWM <-fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrixSWM2012-14ForROMSComp.csv")

#read in biophysical data for KERNEL FITTING
FullBiophysMat <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2012-14ForROMSComp.csv")
AnnualBiophysMat2012 <- fread(file="~/oceanography/script_output/ROMSDataTables20210701_BioPhysParentageMatrix2012ForROMSComp.csv")
AnnualBiophysMat2013 <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2013ForROMSComp.csv")
AnnualBiophysMat2014 <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2014ForROMSComp.csv")
MonsoonBiophysMatNEM <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrixNEM2012-14ForROMSComp.csv")
MonsoonBiophysMatSWM <- fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrixSWM2012-14ForROMSComp.csv")

Warning message:
“package ‘R.utils’ was built under R version 3.6.3”Warning message:
“package ‘R.oo’ was built under R version 3.6.3”Warning message:
“package ‘R.methodsS3’ was built under R version 3.6.3”

In [ ]:


#fit the kernels, get the data together
par_data2012 <- list(Distances=Distances, Assignments=AnnualBiophysMat2012, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=AnnualBiophysMat2014, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=MonsoonBiophysMatNEM, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(SiteIndex[site %in% SurveyData[, site], index]), #if CAI is it's own site- site %in% AllYearsRec[, dest]
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))



In [ ]:

#make a summary table for all time frames of simulation data
SimKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 7))
setDT(SimKernels)
setnames(SimKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90"))
#SimKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[1] <- "annual"
SimKernels$time_id[1] <- "2012"
SimKernels$k[1] <-  BestK2012
SimKernels$theta[1] <- BestTheta2012
SimKernels$mdd[1] <- MDD2012
SimKernels$med[1] <- Med2012
SimKernels$dist90[1] <- Dist90_2012

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[2] <- "annual"
SimKernels$time_id[2] <- "2013"
SimKernels$k[2] <-  BestK2013
SimKernels$theta[2] <- BestTheta2013
SimKernels$mdd[2] <- MDD2013
SimKernels$med[2] <- Med2013
SimKernels$dist90[2] <- Dist90_2013

BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[3] <- "annual"
SimKernels$time_id[3] <- "2014"
SimKernels$k[3] <-  BestK2014
SimKernels$theta[3] <- BestTheta2014
SimKernels$mdd[3] <- MDD2014
SimKernels$med[3] <- Med2014
SimKernels$dist90[3] <- Dist90_2014

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[4] <- "interannual"
SimKernels$time_id[4] <- "2012-4"
SimKernels$k[4] <-  BestK2012_4
SimKernels$theta[4] <- BestTheta2012_4
SimKernels$mdd[4] <- MDD2012_4
SimKernels$med[4] <- Med2012_4
SimKernels$dist90[4] <- Dist90_2012_4

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[5] <- "season"
SimKernels$time_id[5] <- "NEM"
SimKernels$k[5] <-  BestKNEM
SimKernels$theta[5] <- BestThetaNEM
SimKernels$mdd[5] <- MDDNEM
SimKernels$med[5] <- MedNEM
SimKernels$dist90[5] <- Dist90_NEM

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[6] <- "season"
SimKernels$time_id[6] <- "SWM"
SimKernels$k[6] <-  BestKSWM
SimKernels$theta[6] <- BestThetaSWM
SimKernels$mdd[6] <- MDDSWM
SimKernels$med[6] <- MedSWM
SimKernels$dist90[6] <- Dist90_SWM

#fwrite(SimKernels, file="~/oceanography/script_output/KernelFits/SimulationKernelFits.csv")


SimKernels

kernel2012_14

In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=par_dataSWM$Distances, Assignments=par_dataSWM$Assignments, Sampled_reefs=par_dataSWM$Sampled_reefs, Reef_sizes=par_dataSWM$Reef_sizes, Adult_sample_proportions=par_dataSWM$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
#write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysicalSWM.csv", row.names=T, quote=FALSE)

In [ ]:
##lay out all the pieces
#sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)])
#pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
#BioPhysMat <- as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]) #source normalized biophysical connectivity matrix. In Eqn. S3.4, this is m ajt/r a (*should it be r at? As in all particles released in time period t?)
#prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function
#unassigned_vec <- as.matrix(GenMat2012_4[nrow(GenMat2012_4),])#from genetic parentage data- a vector of the number of unassigned recruits at each destination reef in the system- we only have this for all sampled reefs.... what should the dimensions be?*
#Assignments <- GenMat2012_4[1:nrow(GenMat2012_4)-1,]
#
Data2012_4 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012_4[nrow(GenMat2012_4),]))

Data2012 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
 Data2014 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

 DataNEM <- list(BioPhysMat=as.matrix(MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM <- list(BioPhysMat=as.matrix(MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))

#calculate likelihood of our observed data given the biophysical simulation data- we want the highest likelihood, which is the smallest negative log likelihood
data.table(time_frame= c("interannual", "annual", "monsoonal"), neg_LL=c(neg_LL_biophys(Data2012_4), neg_LL_biophys(Data2012) + neg_LL_biophys(Data2013) + neg_LL_biophys(Data2014), neg_LL_biophys(DataNEM) + neg_LL_biophys(DataSWM)))[order(-neg_LL)]


Data2012_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
Data2014_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

neg_LL_biophys(Data2012_test)+neg_LL_biophys(Data2013_test)+neg_LL_biophys(Data2014_test)

 DataNEM_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatNEM[1:nrow(GenMatNEM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatSWM[1:nrow(GenMatSWM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))
neg_LL_biophys(DataNEM_test)+neg_LL_biophys( DataSWM_test)

sum(colSums(MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]))
sum(colSums(MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]))

